### Importing prerequisite libraries

In [11]:
import re
from bs4 import BeautifulSoup
import requests
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import time
import random 
import pandas as pd

### Function for scraping the data

In [12]:
def scrap(url):
    global books
    
    #sending a get request to the URL
    page=requests.get(url,headers=headers)
    soup=BeautifulSoup(page.text,'lxml')
    print(url) 

    #Extracting the novel details
    n=soup.find_all("span",itemprop="name",role='heading')
    au = soup.find_all('div',class_='authorName__container')
    r=soup.find_all("span",class_="minirating")
    sv=soup.find_all('span',class_='smallText uitext')
    E=soup.find_all("a",class_="bookTitle")
 

    for i in range(0,100):
        print(i)

        book = {
                "Novel_Name": None, 
                "Series_Name": None, 
                "Author_Name": None, 
                "Genre": None,
                "Average_Rating": None, 
                "No_of_Ratings": None, 
                "No_of_Reviews": None,
                "Score": None,
                "No_of_Votes": None,
                "Publish_Year": None,
                "Pages": None
                }
        
        
        #Extracting novel name and seires name 
        if "(" in str(n[i]):
            x=n[i].text.split("(")
            book["Novel_Name"] = x[0]
            if "#" in str(n[i]):
                x=re.findall("\((.*)\s#",n[i].text)[0]
                if x.endswith(","):
                    book["Series_Name"] = x[0:-1]
                else:  
                    book["Series_Name"] = x
            else:
                element = re.findall("\((.*)\s",n[i].text)
                if element:
                    book["Series_Name"] = element[0]
        else:
            book["Novel_Name"] = n[i].text

        
        #Extracting author name
        b=au[i].text.split("\n")[1]
        if "(" in b: 
            book["Author_Name"] = b.split("(")[0]
        else: 
            book["Author_Name"] = b

        #Extracting avg ratings and number of ratings
        book["Average_Rating"] = re.findall("\s(\d+\.\d+)\savg",r[i].text)[0]
        book["No_of_Ratings"] = re.findall("\—\s(.*)\srating",r[i].text)[0]
        
        #Extracting score and number of votes
        book["Score"] = re.findall("score:\s(.*),",sv[i].text)[0]
        book["No_of_Votes"] = re.findall("and\s(.*)\sp",sv[i].text)[0]
        
        #Determining the genre based on the URL
        book["Genre"] = list_of_genre[URL.index(url_)]

        #Following the link to the book details for aditional book information
        d='https://www.goodreads.com'+E[i].get('href')
        page=requests.get(d,headers=headers)
        soup=BeautifulSoup(page.text,'lxml')  

        #Extracting publish year 
        p=soup.find_all("div",class_="FeaturedDetails")
        if p:
            element = re.findall("ublished\s(\w+\s\d+,\s\d+)",p[0].text)
            if element:
                book["Publish_Year"] = element[0]
                
                
        #Extracting number of reviews
        rev=soup.find("span",class_="u-dot-before")
        if rev:
            book["No_of_Reviews"] = rev.text.split("\xa0")[0]

        #Extracting number of pages
        pa=soup.find_all('p',{'data-testid':'pagesFormat'})    
        if pa:
            element = re.findall("(.*)\spage",pa[0].text)
            if element:
                book["Pages"] = element[0]
        
        books.append(book)
        
        #sleep to avoid overwhelming the server
        time.sleep(random.uniform(1,14))

### Scraping the data

In [13]:
#List of genres
list_of_genre=["Action-Adventure","Epic Fantasy","Urban Fantasy","Historical","Crime/Mystery","Romance","Suspence/Thriller","Science Fiction","Horror/Paranormal"]
#List of URL corresponding to the genres
URL=['https://www.goodreads.com/list/show/514.Best_Action_Adventure_Novels','https://www.goodreads.com/list/show/50','https://www.goodreads.com/list/show/51','https://www.goodreads.com/list/show/3116.Best_historical_fiction_novels','https://www.goodreads.com/list/show/11.Best_Crime_Mystery_Books','https://www.goodreads.com/list/show/12362.All_Time_Favorite_Romance_Novels','https://www.goodreads.com/list/show/348.Thrillers','https://www.goodreads.com/list/show/19341.Best_Science_Fiction','https://www.goodreads.com/list/show/135.Best_Horror_Novels']

books=[]

#HTTP Header to simulate a browser request 
headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}   

for url_ in URL:
    url=[]
    for i in range(1,9):
        if i==1:
            url1=url_
        else:
            url1=url_+"?page="+str(i) 
        url.append(url1)
        
    #Handling concurrent requests for each URL
    with ThreadPoolExecutor() as executor:
        executor.map(scrap,url) 


https://www.goodreads.com/list/show/514.Best_Action_Adventure_Novels?page=5
0
https://www.goodreads.com/list/show/514.Best_Action_Adventure_Novels?page=8
0
https://www.goodreads.com/list/show/514.Best_Action_Adventure_Novels?page=3
0
https://www.goodreads.com/list/show/514.Best_Action_Adventure_Novels
0
https://www.goodreads.com/list/show/514.Best_Action_Adventure_Novels?page=7
0
https://www.goodreads.com/list/show/514.Best_Action_Adventure_Novels?page=6
0
https://www.goodreads.com/list/show/514.Best_Action_Adventure_Novels?page=2
0
https://www.goodreads.com/list/show/514.Best_Action_Adventure_Novels?page=4
0
1
1
1
1
1
1
1
2
2
2
1
2
2
2
2
3
3
2
3
3
3
3
3
4
4
4
5
4
4
4
3
5
5
6
4
5
5
5
7
6
6
8
4
6
6
6
5
9
7
7
5
7
7
7
6
8
8
8
7
8
8
9
6
10
8
10
9
9
11
9
9
7
10
12
9
11
10
8
10
11
11
13
10
10
12
11
9
11
14
11
12
12
10
12
13
15
12
13
12
13
11
13
16
13
13
14
14
17
12
14
13
14
14
18
15
15
14
15
15
15
16
15
19
14
20
16
17
16
16
16
16
21
17
17
17
15
17
18
17
18
18
22
19
18
16
19
19
18
20
23
18
17

93
90
92
98
97
99
98
91
94
93
99
95
92
94
96
93
95
96
94
97
97
98
95
98
99
96
99
97
98
99
https://www.goodreads.com/list/show/3116.Best_historical_fiction_novels?page=5
0
https://www.goodreads.com/list/show/3116.Best_historical_fiction_novels
0
https://www.goodreads.com/list/show/3116.Best_historical_fiction_novels?page=8
0
https://www.goodreads.com/list/show/3116.Best_historical_fiction_novels?page=4
0
https://www.goodreads.com/list/show/3116.Best_historical_fiction_novels?page=2
0
https://www.goodreads.com/list/show/3116.Best_historical_fiction_novels?page=7
0
https://www.goodreads.com/list/show/3116.Best_historical_fiction_novels?page=3
0
https://www.goodreads.com/list/show/3116.Best_historical_fiction_novels?page=6
0
1
1
1
1
1
2
1
1
2
1
2
2
2
3
2
2
3
3
3
3
2
4
4
3
3
4
5
5
4
5
4
4
3
4
6
6
6
5
5
5
5
7
4
86
7
7

6
6
6
5
8
9
8
7
7
7
7
6
8
10
9
8
9
8
8
7
9
10
11
10
9
9
9
10
8
12
10
11
11
11
10
9
10
12
10
13
11
12
12
11
11
11
13
13
14
12
14
12
14
1512

13
12
13
15
15
13
13
13
16
14
14
14

82
81
80
75
81
79
83
75
84
76
82
76
80
82
77
76
81
85
83
82
78
81
77
86
83
77
84
78
78
83
82
79
79
85
84
87
79
80
84
86
88
83
80
80
85
89
81
87
81
86
82
85
90
84
81
88
82
83
87
85
84
86
83
82
86
85
91
88
84
89
87
89
86
87
83
92
88
85
90
90
88
87
91
86
93
84
91
89
89
94
92
90
87
85
88
90
92
95
91
96
91
93
88
89
93
86
92
92
94
90
87
89
93
97
93
94
90
95
94
98
91
88
95
91
94
96
99
89
95
92
92
97
96
90
98
95
93
96
99
93
97
96
91
94
98
94
97
97
95
95
98
92
98
99
93
99
96
96
99
97
94
97
98
95
99
98
96
99
97
98
99
https://www.goodreads.com/list/show/348.Thrillers?page=5
0
https://www.goodreads.com/list/show/348.Thrillers?page=7
0
https://www.goodreads.com/list/show/348.Thrillers?page=8
0
https://www.goodreads.com/list/show/348.Thrillers
0
https://www.goodreads.com/list/show/348.Thrillers?page=3
0
https://www.goodreads.com/list/show/348.Thrillers?page=4
0
https://www.goodreads.com/list/show/348.Thrillers?page=6
0
1
1
1
1
1
2
2
1
1
2
3
2
2
2
3
2
3
3
3
4
4
3
4
https://www.goodreads.com/list/show

77
75
78
77
83
78
79
82
78
84
79
78
79
76
79
79
80
80
85
83
81
86
77
84
80
80
81
87
85
80
82
78
81
88
82
81
83
81
79
86
84
82
82
83
89
82
87
83
83
80
83
90
85
84
88
85
84
84
91
86
81
89
84
87
92
85
85
86
82
88
85
90
86
87
83
86
93
87
91
89
86
84
87
88
88
92
94
90
88
85
89
93
95
89
91
87
89
86
90
90
94
88
87
92
96
91
89
90
91
88
97
95
92
93
92
91
96
89
93
94
90
98
93
92
95
90
94
94
99
91
96
95
97
93
91
95
98
94
92
97
96
98
96
92
95
99
97
97
93
99
98
93
94
96
98
99
99
95
97
94
96
98
97
95
99
96
98
97
98
99
99


In [14]:
#Dataframe after scraping 
novel = pd.DataFrame(books)
novel

,Novel_Name,Series_Name,Author_Name,Genre,Average_Rating,No_of_Ratings,No_of_Reviews,Score,No_of_Votes,Publish_Year,Pages
0,Clifford's War: The Bluegrass Battleground,None,J. Denison Reed,Action-Adventure,4.42,33,16,398,4,"May 27, 2021",280
1,Jurassic Park,Jurassic Park,Michael Crichton,Action-Adventure,4.09,"944,442","19,172","30,359",307,"November 7, 1990",466
2,Sunburst,Sky King Ranch,Susan May Warren,Action-Adventure,4.43,"1,467",335,191,2,"June 1, 2022",368
3,The Paris Apartment,None,Lucy Foley,Action-Adventure,3.67,"473,386","40,240",296,3,"February 22, 2022",360
4,Dempsey,Tier One,Brian Andrews,Action-Adventure,4.58,"1,708",86,"1,260",13,"February 21, 2023",350
...,...,...,...,...,...,...,...,...,...,...,...
7035,They Thirst,None,Robert McCammon,Horror/Paranormal,3.91,"12,688",564,"3,615",46,"January 1, 1981",412
7036,Snap,None,Steven Bredice,Horror/Paranormal,3.40,43,11,200,2,"January 1, 2012",200
7037,The Traveling Vampire Show,None,Richard Laymon,Horror/Paranormal,3.83,"8,813",645,"3,577",40,"April 1, 2000",400
7038,WISHBONE...Be Careful What You Wish For,None,Brooklyn Hudson,Horror/Paranormal,3.73,764,94,200,2,"February 20, 2012",365


In [15]:
#Saving the DataFrame as csv file
novel.to_csv(r"Data Sets\scraped_novel_dataset.csv", encoding="utf-8-sig", index=False, quotechar='"')